## MONGODB-PROJECT: GEOSPATIAL QUERIES

!pip3 install pymongo

In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

★ Defino una funcion para conectarme a MongoDB

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

★ Le digo el database y la coleccion a la que quiero acceder y conectarme

In [3]:
db, coll = connectCollection('companies','companies')

★ Busco todas las compañias que esten en la industria "games_video"

In [4]:
gaming_companies = list(coll.find({'category_code':"games_video"}))
# len(gaming_companies) == 1083

★ Filtro el db con una query a mongodb. Le pido que me devuelva todas las compañías que no hayan quebrado y aquellas que tengan más de una oficina, que me seleccione todas.

In [5]:
company2 = list(coll.find({"$and": [{"deadpooled_year":None},{"$where":'this.offices.length >= 1'}]}))

In [6]:
len(company2)

12957

★ Creo un diccionario de una lista a la que le voy metiendo todos los valores con los que me quiero quedar.

In [7]:
merged_offices = []
for c in company2:
    offices = c.get('offices',[])
    for office in offices:
        merged_office = dict()
        raised_amount=0
        merged_office["ref"] = str(c.get('_id',0))
        merged_office["name"]=c.get('name',0)
        merged_office["category"]=c.get('category_code',0)
        merged_office["description"]=c.get('description',0)
        merged_office["latitude"]=office.get('latitude',0)
        merged_office["longitude"]=office.get('longitude',0)
        merged_office["founded_year"]=c.get('founded_year',0)
        merged_office["total_money_raised"]=c.get('total_money_raised',0)
        for rounds in range(len(c.get('funding_rounds'))):
            r=c.get('funding_rounds')[rounds].get('raised_amount',0)
            if r !=None:
                raised_amount+=r
        merged_office["raised_amount"]=raised_amount
        if len(c.get('funding_rounds'))>0:
            merged_office["currency"]=c.get('funding_rounds')[0].get('raised_currency_code',0)
        else:
            merged_office["currency"]=0
        merged_offices.append(merged_office)
        merged_office['address'] = office.get('address1', 0)
        merged_office['zip_code'] = office.get('zip_code', 0)
        merged_office['city'] = office.get('city', 0)

★ Creo un DataFrame (df) con toda esa información

In [8]:
data = merged_offices
df = pd.DataFrame(data)
df.shape

(15848, 13)

★ Asigno a la columna 'currency' los simbolos de la moneda que salgan en la columna 'raised_amount'. De esta manera, evito tener "0" en 'currency'.

In [9]:
currencies = {"$":"USD", "C$":"CAD", "£":"GBP", "€":"EUR", "¥":"JPY", "kr":"SEK"}
for key,value in currencies.items():
    df.loc[(df['total_money_raised'].str.startswith(key)),'currency']=value

## Filtro de 1M

★ Ahora que ya tengo toda la columna 'currency' con sus respectivos valores, creo un dataframe (df_currency) que me filtre por aquellas empresas en las que el 'raised_amount' sea > 1M pero teniendo en cuenta el tipo de moneda que sea, i.e., euros, dolares, dolares canadienses, coronas suecas, etc.

In [10]:
df_currency=df[(((df['raised_amount']>=1000000) & (df['currency']=='USD'))|((df['raised_amount']>=1320000) & (df['currency']=='CAD'))|((df['raised_amount']>=900000) & (df['currency']=='EUR'))|((df['raised_amount']>=770000) & (df['currency']=='GBP'))|((df['raised_amount']>=108560000) & (df['currency']=='JPY'))|((df['raised_amount']>=9640000) & (df['currency']=='SEK')))]

In [20]:
df_currency

,ref,name,category,description,latitude,longitude,founded_year,total_money_raised,raised_amount,currency,address,zip_code,city
2,52cdef7c4bab8bd675297d8d,Digg,news,user driven social content website,37.764726,-122.394523,2004.0,$45M,45000000.0,USD,135 Mississippi St,94107,San Francisco
3,52cdef7c4bab8bd675297d91,Geni,web,Geneology social network site,34.090368,-118.393064,2006.0,$16.5M,16500000.0,USD,9229 W. Sunset Blvd.,90069,West Hollywood
4,52cdef7c4bab8bd675297d95,StumbleUpon,web,Content discovery service,37.775196,-122.419204,2002.0,$18.5M,18500000.0,USD,,,San Francisco
5,52cdef7c4bab8bd675297d95,StumbleUpon,web,Content discovery service,NaN,NaN,2002.0,$18.5M,18500000.0,USD,,,New York City
7,52cdef7c4bab8bd675297d96,Gizmoz,web,Photo animation,37.484130,-122.169472,2003.0,$18.1M,18100000.0,USD,None,None,Menlo Park
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15828,52cdef7f4bab8bd67529c6de,Cantimer,None,,37.484616,-122.181753,2006.0,$2M,2000000.0,USD,3700 Haven Court,94025,Menlo Park
15829,52cdef7f4bab8bd67529c6e0,Celestial Semiconductor,semiconductor,,37.386417,-121.929170,2004.0,$2M,2000000.0,USD,"2635 North 1st Street, Suite 150",95134,San Jose
15830,52cdef7f4bab8bd67529c6e3,InSound Medical,biotech,Hearing Devices,37.512038,-121.998513,NaN,$11M,11000000.0,USD,39660 Eureka Drive,94560,Newark
15831,52cdef7f4bab8bd67529c6e4,Getyoo,mobile,NFC Events Solutions,50.839970,4.346472,2009.0,€1.13M,1130000.0,EUR,"Rue des Tanneurs, 60a",1000,Brussels


★ Aquí simplemente estoy comprobando que me ha sustituido los "0" de la columna 'currency' por el tipo de moneda que le corresponda.

In [11]:
#set(df['currency'])

In [12]:
#df_currency['longitude']

## GOOGLE API 

## Rellenar longitudes/latitudes nulas dandole a la googleAPI la dirección.

In [13]:
import os
import requests
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
def googleAPI(params):
    authToken = os.getenv("GOOGLE_API")
    url = "https://maps.googleapis.com/maps/api/geocode/json?address={}&key={}".format(params,authToken)
    #print("Requesting url {}...".format(url))
    res = requests.get(url).json()
    #print("Done")
    return res

In [15]:
def get_lat_long(address):
    coordenadas = googleAPI(address)
    return coordenadas

In [16]:
addresses = list(df_currency['address'])
longitude = list(df_currency['longitude'])
for i in range(len(addresses)):
    if np.isnan(longitude[i]):
        if addresses[i]!= "":
            dat=get_lat_long(addresses[i])
            if dat['status'] != 'ZERO_RESULTS' and dat['status']!='REQUEST_DENIED' and dat['status']!='INVALID_REQUEST':
                df_currency.loc[(df_currency['address']==addresses[i]),'latitude'] = dat['results'][0]['geometry']['location']['lat']
                df_currency.loc[(df_currency['address']==addresses[i]),'longitude'] = dat['results'][0]['geometry']['location']['lng']

Done
Done


/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [17]:
df_currency.shape

(4824, 13)

★ Elimino todos las filas en las que la longitud/latitud sea nula, es decir, todos aquellos campos que la googleAPI no haya podido rellenar ya sea porque no tenian direccion (address) o porque la API diera alguno de los tres errores que he puesto en el bucle anterior. Mi df_currency tiene 324 nulos entre las columnas latitud&longitud, asi que procedo a eliminarlos.

In [19]:
df_currency[(df_currency['longitude'].isnull() & df_currency['latitude'].isnull())].shape

(324, 13)

In [23]:
dfinitivo_1M = df_currency.dropna(subset=['longitude', 'latitude'])
dfinitivo_1M.shape

(4500, 13)

★ Defino la funcion getLocation() que me crea un diccionario en formato GeoIndex 
y mete las longitudes y latitudes de todas las oficinas de mi df

In [25]:
def getLocation(long, lat):
    loc = {
        'type':'Point',
        'coordinates':[float(long), float(lat)]
    }
    return loc

In [26]:
df_long_lat = dfinitivo_1M[['longitude', 'latitude']] 
location = []
for long,lat in df_long_lat.values:
    location.append(getLocation(long, lat))

★ Creo la columna 'GeoIndex' en mi dfinitivo_1M original con el formato {type,coordinates}

In [27]:
dfinitivo_1M['GeoIndex'] = location

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(4500, 14)

★ Divido mis preferencias en tres dataframes distintos:

➊ Dataframe con todas las compañias que hayan levantado mas de 1M.

In [28]:
dfinitivo_1M.shape

(4500, 14)

➋ Dataframe con todas las compañias que han levantado mas de 1M y que además han sido creadas en los ultimos 10 años, i.e., las compañias que queremos tener cerca.

In [29]:
startups = dfinitivo_1M[dfinitivo_1M['founded_year']>=2009]
startups.shape

(141, 14)

➌ Dataframe con todas las compañias que nacieron antes de 2009 (sin tener en cuenta los $ que hayan levantado), i.e., las que queremos tener lejos.

In [30]:
old = df[df['founded_year']<=2009]
old.shape

(13239, 13)

★ Convierto los 3 dataframes a json para importarlos con mongod desde la terminal 
como collections a MongoCompass

Import Collection: mongoimport --db <database> --collection <collection_name> --jsonArray <file_name.json>

Update Collection: mongoimport --db <database> --drop --collection <collection_name> --jsonArray <file_name.json>

In [33]:
dfinitivo_1M.to_json('./companies1M.json', orient="records")

startups.to_json('./startups1M.json', orient="records")

old.to_json('./oldCompanies.json', orient="records")

## STARBUCKS